In [54]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import  ChatPromptTemplate, FewShotPromptTemplate,PromptTemplate ,MessagesPlaceholder


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
    k = 1
)

examples = [
    {
        "movie": "탑건",
        "answer": "🛩️👨‍✈️🔥"
    },
    {
        "movie": "대부",
        "answer": "👨‍👨‍👦🔫🍝"
    }
]

example_prompt = PromptTemplate.from_template(
    "Human: Express the movie {movie} using 3 emojis. AI: {answer}"
)

few_shot_prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: Express the movie {movie} using 3 emojis. ",
    input_variables=["movie"]
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI that summarizes movies using exactly three emojis."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie}"),
])


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)



In [55]:
invoke_chain("탑건")

content='✈️🔥🕶️'


In [56]:
invoke_chain("대부")

content='🇮🇹🔫💰'


In [57]:
invoke_chain("The Avengers")

content='🦸\u200d♂️💥🌍'


In [64]:
invoke_chain("Inception")

content='🧠🌀🌌'


In [63]:
def check_movie():
    history = load_memory(_)
    print("앞의 영화 : ")
    print(history)

check_movie()

앞의 영화 : 
[HumanMessage(content='탑건'), AIMessage(content='✈️🔥🕶️'), HumanMessage(content='대부'), AIMessage(content='🇮🇹🔫💰'), HumanMessage(content='The Avengers'), AIMessage(content='🦸\u200d♂️💥🌍'), HumanMessage(content='Inception'), AIMessage(content='🧠🌀🌌')]
